In [1]:
import cv2
import pytesseract
from pytesseract import Output
import pandas as pd

In [8]:
img = cv2.imread('data/invoice.jpg')

d = pytesseract.image_to_data(img, output_type=Output.DICT)
print(d.keys())

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


In [9]:
df = pd.DataFrame(d)

In [ ]:
n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 60:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('img', img)
cv2.waitKey(0)

In [7]:
import requests
import time

# Define the URL for the upload endpoint
upload_url = "http://localhost:8000/upload/"

# Define the file path
file_path = "data/invoice.jpg"

# Read the file and send a POST request to the upload endpoint
with open(file_path, "rb") as file:
    response = requests.post(upload_url, files={"file": file})

# Check if the upload was successful
if response.status_code == 200:
    data = response.json()
    task_id = data["task_id"]
    print(f"File uploaded successfully. Task ID: {task_id}")
else:
    print(f"Failed to upload file. Status code: {response.status_code}")
    exit()



File uploaded successfully. Task ID: 92e0a431-8ab2-4a11-8ea8-d63209c7dd9d


In [8]:

# Define the URL for the result endpoint
result_url = f"http://localhost:8000/result/{task_id}"

# Poll the result endpoint until the processing is done
while True:
    result_response = requests.get(result_url)
    if result_response.status_code == 200:
        result_data = result_response.json()
        if result_data["state"] == "SUCCESS":
            print("OCR result:", result_data["results"])
            break
        elif result_data["state"] == "PENDING":
            print("Processing is still in progress. Retrying in 5 seconds...")
            time.sleep(5)
        else:
            print("Unexpected state:", result_data["state"])
            break
    else:
        print(f"Failed to retrieve result. Status code: {result_response.status_code}")
        break

OCR result: [{'level': 1, 'page_num': 1, 'block_num': 0, 'par_num': 0, 'line_num': 0, 'word_num': 0, 'left': 0, 'top': 0, 'width': 835, 'height': 1150, 'conf': -1, 'text': ''}, {'level': 2, 'page_num': 1, 'block_num': 1, 'par_num': 0, 'line_num': 0, 'word_num': 0, 'left': 656, 'top': 41, 'width': 119, 'height': 11, 'conf': -1, 'text': ''}, {'level': 3, 'page_num': 1, 'block_num': 1, 'par_num': 1, 'line_num': 0, 'word_num': 0, 'left': 656, 'top': 41, 'width': 119, 'height': 11, 'conf': -1, 'text': ''}, {'level': 4, 'page_num': 1, 'block_num': 1, 'par_num': 1, 'line_num': 1, 'word_num': 0, 'left': 656, 'top': 41, 'width': 119, 'height': 11, 'conf': -1, 'text': ''}, {'level': 5, 'page_num': 1, 'block_num': 1, 'par_num': 1, 'line_num': 1, 'word_num': 1, 'left': 656, 'top': 41, 'width': 119, 'height': 11, 'conf': 75, 'text': 'http://mrsinvoice.com'}, {'level': 2, 'page_num': 1, 'block_num': 2, 'par_num': 0, 'line_num': 0, 'word_num': 0, 'left': 680, 'top': 64, 'width': 67, 'height': 54, 'co

In [5]:
result_data

{'state': 'PENDING',
 'status': 'in_progress',
 'file_name': 'invoice.jpg',
 'start_datetime': '2024-05-27T22:04:51.975492',
 'end_datetime': None,
 'width': None,
 'height': None}

In [6]:
import requests

# Define the URL for the health endpoint
health_url = "http://localhost:8000/health"

# Send a GET request to the health endpoint
response = requests.get(health_url)

# Check if the request was successful
if response.status_code == 200:
    health_status = response.json()
    print(f"Health Check: {health_status}")
else:
    print(f"Failed to get health status. Status code: {response.status_code}, Response: {response.text}")


Health Check: {'status': 'healthy'}


In [1]:
import cv2
import numpy as np

img = cv2.imread("data/invoice.jpg")

# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

In [2]:
imgs = get_grayscale(img)
imgs = thresholding(imgs)
cv2.imshow('img', imgs)